In [1]:



import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

# 绘制前十张图像
def data_show(dataAll,label):
    plt.figure() # 新建画布，防止堆叠
    for i in range(1,11):
        plt.subplot(1,10,i)
        data = dataAll[i, :]    #i为第几个数据
        data = data.reshape(28, 28)  #将数据转为（28，28）维度
        img = Image.fromarray(data)  #转为图片
        plt.imshow(img)
    plt.title(label)
    plt.show()

# 加载数据
def data_load(dataName):
    path = "data/quick_draw_data/"
    dataAll = np.load(path + dataName+"/"+dataName+".npy")
#     data_show(dataAll, dataName)
    return dataAll

In [2]:
# 调用方法读取数据
apple = data_load("apple")
ambulance = data_load("ambulance")
bear = data_load("bear")
bicycle = data_load("bicycle")
bird = data_load("bird")
bus = data_load("bus")
cat = data_load("cat")
foot = data_load("foot")
owl = data_load("owl")
pig = data_load("pig")

# 构建CNN（例如Alex net）作为10-分类模型

Alexnet由八层网络构成，其中包括5层卷积层和3层全连接层。卷积层包含了卷积、激活、池化、局部归一化几个过程，全连接层包括了droupout、激活、全连接几个过程。

In [3]:
import torch
from torch import nn

In [4]:
# 构建AlexNet网络变体作为分类器
class Net(nn.Module):
    def __init__(self,num_classes=10,init_weights=False):
        super(Net, self).__init__()
        
        self.features = nn.Sequential(
            # 特征提取层，padding填充
#             输出大小 = ((输入大小 - 卷积核大小) / 步长) + 1
            nn.Conv2d(1,20,kernel_size=3,stride=3,padding=2), # Input[28，28，1]    output[10,10,20]
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3,stride=2),
            
            nn.Conv2d(20, 40, kernel_size=3, stride=1,padding=1),  # input[10,10,20]  output[8,8,40]
            nn.ReLU(inplace=True),
            
            nn.Conv2d(40, 40, kernel_size=3, stride=1,padding=1),  # input[8,8,40]  output[6,6,40]
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.classifier = nn.Sequential(
            # 全连接层
            nn.Dropout(p=0.5),  # p:神经元随机失活的比例
            
            nn.Linear(6*6*40,500),
            nn.ReLU(inplace=True),
            
            nn.Linear(500,num_classes)  # num_classes：分类的类别数
        )

        if init_weights:
            self._initialize_weights()
    
    # 前向过程
    def forward(self,x):
        x = self.features(x)  # 特征提取层
        x = torch.flatten(x,start_dim=1)
        x = self.classifier(x)  # 全连接+分类
        return x

    # 初始化权重
    def _initialize_weights(self):
        # 遍历整个modules
        for m in self.modules():
            # 发现有nn.Conv2d这个结构
            if isinstance(m,nn.Conv2d):
                # 凯明初始化权重
                nn.init.kaiming_normal_(m.weight,mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias,0)
            # 发现有nn.Linear，则使用正态分布初始化函数
            elif isinstance(m,nn.Linear):
                nn.init.normal_(m.weight,0,0.01)
                nn.init.constant_(m.bias,0)


# 训练模型
每类数据按照6:3:1方式划分训练集、验证集、测试集对模型进行训练，绘制训练过程中损失函数和预测acc曲线（在同一幅图中）

In [5]:
# 数据预处理

# 数据划分
def data_split(arr):
    # 按比例6:3:1随机划分数组

    # 随机打乱数组顺序
    np.random.shuffle(arr)

    # 计算划分点的索引
    split1 = int(len(arr) * 0.7)
    split2 = int(len(arr) * 0.9)

    # 按比例6:3:1划分数组
    split_arr = np.split(arr, [split1, split2])
    return split_arr


label_names=['ambulance','apple','bear','bicycle','bird','bus','cat','foot','owl','pig']

# 存放不同类别的训练集、验证集、测试集
train = {}
val = {}
test = {}

for i in range(0,10):
    arr = locals()[label_names[i]]  # 依据标签名定位数据
    train[i],val[i],test[i] = data_split(arr)  # i即为标签索引


In [19]:
from torch.nn.functional import interpolate
from torch import optim
# from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms

ImportError: cannot import name '_get_cpp_backtrace' from 'torch._C' (D:\Anaconda3\envs\torch\lib\site-packages\torch\_C.cp310-win_amd64.pyd)

In [16]:
def transf(data):
    data = data.reshape(28, 28)#将数据转为（28，28）维度
    img = Image.fromarray(data) #转为图片
    # 定义变换
    transform = transforms.Compose([
        transforms.ToTensor(),  # 转换为 torch.Tensor
    ])

    # 进行变换
    tensor_image = transform(img)
    return tensor_image

train_tensor = {}
for i in range(10):
    for j in range(len(train[i])):
        train_tensor[i][j] = transf(train[i][j])
        
val_tensor = {}
for i in range(10):
    for j in range(len(val[i])):
        val_tensor[i][j] = transf(val[i][j])
        
test_tensor = {}
for i in range(10):
    for j in range(len(test[i])):
        test_tensor[i][j] = transf(test[i][j])

NameError: name 'transforms' is not defined

In [11]:
# 计算模型准确度
def test(model):
    # 批量数目
    batch_size = 10
    # 预测正确个数
    correct = 0
    
    # 从验证集加载200个数据进行验证
    for i in range(10):
        for j in np.random.sample(range(0,len(test[0])),20):
            # 预测
            prediction = model(test_tensor[i][j]))
            # 将预测值中最大的索引取出，其对应了不同类别值
            predicted = torch.max(prediction.data, 1)[1]
            # 获取准确个数
            if predict == i:
                correct+=1
    acc = correct / 200        
#     print('准确率: %d' % (correct / 200)) # 因为总共500个测试数据
    return acc

In [12]:
# 训练过程
def model_train():
    batch_size = 16     # 批量训练大小
    model = Net() # 创建模型
    
    # 定义损失函数
    loss_func = nn.CrossEntropyLoss()
    # 定义优化器
    optimizer = optim.Adam(params=model.parameters(),lr=0.0002)
    
    # 定义损失和准确度
    loss = []
    acc = []
    
    for n in range(0,10): # 10个epoch
        print("epoch [%d]" % n)
        loss_temp = 0  # 临时变量
        
        for i in range(0,len(train[0])):
            for j in range(0,10):
                # 梯度清零
                optimizer.zero_grad()
                
                # 模型训练
                prediction = model(train_tensor[i][j])
                # 损失值
                loss = loss_func(prediction,j)
                loss_temp += loss.item()
                # 反向传播
                loss.backward()
                # 梯度更新
                optimizer.step()
        if  ((i+1)%100)==0 :
            # 打印一次损失值
            print('loss: %.3f' % (loss_temp/1000))
            loss.append(loss_temp/1000)
            # 打印一次准确度
            acc_temp = test(model)
            acc.append(acc_temp)
    return model,loss,acc

In [13]:
model,loss,acc = model_train()

epoch [0]


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [784]

# 验证模型
从每类数据的测试集中随机抽取10张图片进行预测，输出该图像的标签以及预测概率分布的柱状图

In [ ]:
test(model)